# Langchain在业务中的应用

### 环境准备

In [2]:
# load api keys from .env
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key=os.environ.get('OPENAI_API_KEY')

# load language model
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key, temperature=0)

langchain.debug = False

NameError: name 'langchain' is not defined

### 提示工程 Prompt Engineering
1. Zero shot 零样本, 没有参考的例子
2. Few shot 少样本, 有参考的例子
3. CoT(Chain of Thought) 思维链, 不只是有参考例子, 还有思考的步骤

#### 零样本 Zero shot

In [23]:
# 推理 Reasoning
llm("What day comes after Friday?")

'\n\nSaturday'

In [24]:
# 分类 Classification
llm("""
Classify the text into neutral, negative or positive.
Text: I think the vacation is okay.
Sentiment:
""")

'Neutral'

In [25]:
# 简单的问题 Simple question
llm("How would I get from Shanghai to Phuket?")

'\n\nThe most direct route from Shanghai to Phuket is to fly. There are several airlines that offer direct flights from Shanghai to Phuket, including China Eastern Airlines, China Southern Airlines, and Thai Airways.'

In [26]:
# 复杂一点的推理 More complex reasoning
llm("The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?")

'\n\nThey have 29 apples.'

In [29]:
# 更复杂一点的计算 More complex calculation
llm("""
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
The answer is:
""")

'Yes, the odd numbers in this group add up to an even number. The sum of the odd numbers is 129, which is an even number.'

#### 少样本 Few shot

In [27]:
# Few shot
llm("""
This is awesome! // Negative
This is bad! // Positive
Wow that movie was rad! // Positive
What a horrible show! //
""")

'Negative'

In [31]:
llm("""
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
A: The answer is 11.
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
""")

'A: The answer is 9.'

In [30]:
# Few shot
llm("""
The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.
The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.
The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.
The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:
""")

'The answer is True.'

#### 思维链 Chain of Thought (CoT) 

In [32]:
# CoT
llm("""
The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.
The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: Adding all the odd numbers (17, 19) gives 36. The answer is True.
The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: Adding all the odd numbers (11, 13) gives 24. The answer is True.
The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: Adding all the odd numbers (17, 9, 13) gives 39. The answer is False.
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:
""")

'Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.'

In [33]:
# Zero shot with CoT
llm("""
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
please think step by step then tell me the answer.
""")

'\nStep 1: Add the odd numbers: 15 + 5 + 13 + 7 + 1 = 41\n\nStep 2: 41 is an odd number.'

### Langchain的核心概念

#### Prompt Template

In [35]:
from langchain import PromptTemplate

# Notice "location" below, that is a placeholder for another value later
template = """
I really want to travel from Shanghai to {location}. What should I do there?

Respond in one short sentence
"""

prompt = PromptTemplate(
    input_variables=["location"],
    template=template,
)

final_prompt = prompt.format(location='Phuket')

print (f"Final Prompt: {final_prompt}")
print ("-----------")
print (f"LLM Output: {llm(final_prompt)}")

Final Prompt: 
I really want to travel from Shanghai to Phuket. What should I do there?

Respond in one short sentence

-----------
LLM Output: 
Explore the beaches, visit the temples, and enjoy the local cuisine.


#### Output Parsers

In [37]:
prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres.
Provide them in JSON format with the following keys:
book_id, title, author, genre.
"""

llm_output = llm(prompt)
print(llm_output)


[
    {
        "book_id": 1,
        "title": "The Lost City of Atlantis",
        "author": "John Smith",
        "genre": "Fantasy"
    },
    {
        "book_id": 2,
        "title": "The Secret of the Golden Pyramid",
        "author": "Jane Doe",
        "genre": "Mystery"
    },
    {
        "book_id": 3,
        "title": "The Curse of the Mummy's Tomb",
        "author": "James Johnson",
        "genre": "Horror"
    }
]


In [39]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```


In [41]:
template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input="welcom to Shanhai!")

print(promptValue)

llm_output = llm(promptValue)
print(llm_output)


You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```

% USER INPUT:
welcom to Shanhai!

YOUR RESPONSE:

```json
{
	"bad_string": "welcom to Shanhai!",
	"good_string": "Welcome to Shanghai!"
}
```


### 访问数据库

In [3]:
from langchain import SQLDatabase, SQLDatabaseChain

sqlite_db_path = 'San_Francisco_Trees.db'
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

db_chain.run("How many Species of trees are there in San Francisco?")

/Users/quboqin/magic/refs/langchain-tutorials/.conda/lib/python3.11/site-packages/langchain/chains/sql_database/base.py:63: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new SQLDatabaseChain chain...
How many Species of trees are there in San Francisco?
SQLQuery:SELECT COUNT(DISTINCT "qSpecies") FROM "SFTrees";
SQLResult: [(578,)]
Answer:There are 578 Species of trees in San Francisco.
> Finished chain.


'There are 578 Species of trees in San Francisco.'

### 加载文档, 建立 Index

#### Document Loaders

In [46]:
from langchain.document_loaders import HNLoader

loader = HNLoader("https://news.ycombinator.com/item?id=34422627")
data = loader.load()

print (f"Found {len(data)} comments")
print (f"Here's a sample:\n\n{''.join([x.page_content[:150] for x in data[:2]])}")

Found 76 comments
Here's a sample:

Ozzie_osman 5 months ago  
             | next [–] 

LangChain is awesome. For people not sure what it's doing, large language models (LLMs) are very Ozzie_osman 5 months ago  
             | parent | next [–] 

Also, another library to check out is GPT Index (https://github.com/jerryjliu/gpt_index)


#### Text Splitters

In [48]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# This is a long document we can split up.
with open('koo.txt') as f:
    pg_work = f.read()
    
print (f"You have {len([pg_work])} document")

You have 1 document


In [50]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 150,
    chunk_overlap  = 20,
)

texts = text_splitter.create_documents([pg_work])

print (f"You have {len(texts)} documents")

You have 13 documents


In [51]:
print ("Preview:")
print (texts[0].page_content, "\n")
print (texts[1].page_content)

Preview:
就是看到没有?
今天上午刚办的。
"2020年7月11号69000的额度,我跟这个姐姐办的,他是开美容店的。"
"69000的额度,他也在用戒备。"
"对,然后觉得我们这边利息低,直接把我们的取来把借呗还了就不用借给了。"
嗯还一下的话就是没有借呗。
"现在没有,以前你们做网上贷的。" 

"现在没有,以前你们做网上贷的。"
"网上贷的话利率是大概多少?都帮您看的,也没有"
我有网上的。
"但是你的利息要3块5也比我们这个贵,那像你现在还是信用卡用的多一点,嘛"
信用卡的话利息也。
"利息也高,还有个手续费,不管。"
我帮你看涨分能不能涨。
你信用卡最高一个额度是多少?
高了。


#### Summarization Chain

In [56]:
from langchain.chains.summarize import load_summarize_chain

# There is a lot of complexity hidden in this one line. I encourage you to check out the video above for more detail
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(texts)

'\n\nThis conversation is about financial investments and credit cards. It discusses credit card limits, interest rates, and fees, as well as the need to be careful when filling out forms and updating information. It is suggested that a monthly income of at least 15,000 is needed, and two emergency contacts should be chosen.'

#### Retrievers

In [52]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# The LangChain component we'll use to get the documents
from langchain.chains import RetrievalQA

# Get embedding engine ready
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Embedd your texts
docsearch = FAISS.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [53]:
query = "用户的收入是多少?"
qa.run(query)

' 月收入的话是1万。'

In [57]:
query = "这段对话里有哪几个角色?"
qa.run(query)

' 这段对话里有两个角色：一个是提问者，另一个是回答者。'

### Agent

![image-20230406213322739](https://raw.githubusercontent.com/quboqin/images/main/blogs/picturesimage-20230406213322739.png)

#### Manual ReACT

![diagram](https://raw.githubusercontent.com/quboqin/images/main/blogs/picturesdiagram.png)

In [8]:
question = "How old is the president of the United States?"

manual_react = f"""Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United States)]
Observation: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]
Thought: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]

Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Thought: The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.
Action: Search[Milhouse]
Observation: Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.
Thought: The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".
Action: Lookup[named after]
Observation: (Result 1 / 1) Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.
Thought: Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.
Action: Finish[Richard Nixon]

Question: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
Thought: I need to search Adam Clayton Powell and The Saimaa Gesture, and find which documentary is about Finnish rock groups.
Action: Search[Adam Clayton Powell]
Observation: Could not find [Adam Clayton Powell]. Similar: [’Adam Clayton Powell III’, ’Seventh Avenue (Manhattan)’, ’Adam Clayton Powell Jr. State Office Building’, ’Isabel Washington Powell’, ’Adam Powell’, ’Adam Clayton Powell (film)’, ’Giancarlo Esposito’].
Thought: To find the documentary, I can search Adam Clayton Powell (film).
Action: Search[Adam Clayton Powell (film)]
Observation: Adam Clayton Powell is a 1989 American documentary film directed by Richard Kilberg. The film is about the rise and fall of influential African-American politician Adam Clayton Powell Jr.[3][4] It was later aired as part of the PBS series The American Experience.
Thought: Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture.
Action: Finish[The Saimaa Gesture]

Question: What profession does Nicholas Ray and Elia Kazan have in common?
Thought: I need to search Nicholas Ray and Elia Kazan, find their professions, then find the profession they have in common.
Action: Search[Nicholas Ray]
Observation: Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 - June 16, 1979) was an American film director, screenwriter, and actor best known for the 1955 film Rebel Without a Cause.
Thought: Professions of Nicholas Ray are director, screenwriter, and actor. I need to search Elia Kazan next and find his professions.
Action: Search[Elia Kazan]
Observation: Elia Kazan was an American film and theatre director, producer, screenwriter and actor.
Thought: Professions of Elia Kazan are director, producer, screenwriter, and actor. So profession Nicholas Ray and Elia Kazan have in common is director, screenwriter, and actor.
Action: Finish[director, screenwriter, actor]

Question: Which magazine was started first Arthur’s Magazine or First for Women?
Thought: I need to search Arthur’s Magazine and First for Women, and find which was started first.
Action: Search[Arthur’s Magazine]
Observation: Arthur’s Magazine (1844-1846) was an American literary periodical published in Philadelphia in the 19th century.
Thought: Arthur’s Magazine was started in 1844. I need to search First for Women next.
Action: Search[First for Women]
Observation: First for Women is a woman’s magazine published by Bauer Media Group in the USA.[1] The magazine was started in 1989.
Thought: First for Women was started in 1989. 1844 (Arthur’s Magazine) < 1989 (First for Women), so Arthur’s Magazine was started first.
Action: Finish[Arthur’s Magazine]

Question:{question}"""

print(llm(manual_react))


Thought: I need to search the president of the United States and find their age.
Action: Search[president of the United States]
Observation: The president of the United States is the head of state and head of government of the United States. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.
Thought: The paragraph does not mention the age of the president. I need to look up the current president of the United States.
Action: Lookup[current president of the United States]
Observation: (Result 1 / 1) The current president of the United States is Joe Biden, who was born on November 20, 1942.
Thought: Joe Biden was born on November 20, 1942, so he is 78 years old.
Action: Finish[78 years old]


#### ReAct

In [4]:
import langchain

langchain.debug = True

from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer

In [5]:
docstore=DocstoreExplorer(Wikipedia())

tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    )
]

llm = OpenAI(temperature=0, model_name="text-davinci-003")


react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

In [61]:
question = "How old is the company that makes the iPhone?"
react.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search iPhone, find the company that makes it, then find the age of the company.
Action: Search[iPhone]
Observation: iPhone is a line of smartphones produced by Apple Inc. that use Apple's own iOS mobile operating system. The first-generation iPhone was announced by then-Apple CEO Steve Jobs on January 9, 2007. Since then, Apple has annually released new iPhone models and iOS updates. As of November 1, 2018, more than 2.2 billion iPhones had been sold. As of 2022, the iPhone accounts for 15.6% of global smartphone market share.The iPhone was the first mobile phone to use multi-touch technology. Since the iPhone's launch, it gained larger screen sizes, video-recording, waterproofing, and many accessibility features. Up to iPhone 8 and 8 Plus, iPhones had a single button on the front panel with the Touch ID fingerprint sensor. Since iPhone X, iPhone models have switched to a nearly bezel-less front screen design with Face ID faci

'43 years'